# Çoklu Doğrusal Regresyon

# Model

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
ad = pd.read_csv("Advertising.csv", usecols = [1,2,3,4])
df = ad.copy()
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [3]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

In [4]:
X = df.drop("sales", axis = True)
y = df["sales"]

In [5]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
X = sm.add_constant(X) # constant ekleme

In [6]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state= 42)

In [8]:
df.shape

(200, 4)

In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((160, 4), (40, 4), (160,), (40,))

In [10]:
training = df.copy()

## Statsmodels

In [11]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [12]:
X.head()

,const,TV,radio,newspaper
0,1.0,230.1,37.8,69.2
1,1.0,44.5,39.3,45.1
2,1.0,17.2,45.9,69.3
3,1.0,151.5,41.3,58.5
4,1.0,180.8,10.8,58.4


In [13]:
lm = sm.OLS(y_train, X_train)

In [14]:
model = lm.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sales   R-squared:                       0.896
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                     446.6
Date:                Tue, 30 Aug 2022   Prob (F-statistic):           2.53e-76
Time:                        13:24:17   Log-Likelihood:                -306.64
No. Observations:                 160   AIC:                             621.3
Df Residuals:                     156   BIC:                             633.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9791      0.354      8.427      0.000       2.281       3.677
TV             0.0447      0.002     28.544      0.000       0.042       0.048
radio          0.1892      0.010     19.518      0.000       0.170       0.208
newspaper      0.0028      0.007      0.392      0.696      -0.011       0.017
==============================================================================
Omnibus:                       67.697   Durbin-Watson:                   2.161
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              240.326
Skew:                          -1.627   Prob(JB):                     6.52e-53
Kurtosis:                       8.046   Cond. No.                         471.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
# R-squared ile alakalı değerler bağımlı değişkenin bağımsız değişkenlerce değişiminin açıklanabilirlik değerini verir (%98)
# F-statistic ise modelin anlamlılığını değerlendirir
# Model doğrulama (validasyon) farklı bir konu
# Modellerin tahmin başarısını değerlendirmek için mse ve rmse değerlerine odaklanırız

In [16]:
model.mse_model # hata kareler ortalaması yani mse, TV ve Radio

1238.993097435658

## Sci-Kit Learn model

In [17]:
from sklearn.linear_model import LinearRegression

In [18]:
lm = LinearRegression()
model = lm.fit(X_train, y_train)

In [19]:
model.intercept_

2.979067338122631

In [20]:
model.coef_

array([0.        , 0.04472952, 0.18919505, 0.00276111])

# Tahmin

Model Denklemi:

Sales = 2.97 + TV * 0.044 + Radio * 0.19 + Newspaper * 0.003

Örnek: 30 TV, 10 Radio, 40 Gazete harcaması yapılırsa tahmini satış değeri ?

In [21]:
yeni_veri = [[0], [30], [10],[40]]
yeni_veri = pd.DataFrame(yeni_veri).T #veriyi doğru bir şekilde 3 farklı değişkene verebilmek için df'e çevirdik
# burada ders koduyla farklı olan bir sabitim olduğu için onu dikkate alarak girdi vermem gerekiyordu. 
# ayrıca df çevirme işlemini yapmamızın sebebi matrix çarpımı işlemine hazırlamak

In [22]:
model.predict(yeni_veri)

array([6.32334798])

In [23]:
from sklearn.metrics import mean_squared_error, r2_score

In [24]:
rmse = np.sqrt(mean_squared_error(y_train, model.predict(X_train))) # eğitim hatasını hesaplama

In [25]:
rmse # yukarıda m_s_e fonksiyonunun içine gerçek y değerleri ve predict olan y değerlerini koyut hata kare ortalamasını aldık

1.6447277656443373

In [26]:
rmse_test = np.sqrt(mean_squared_error(y_test, model.predict(X_test))) # test hatasını hesaplama
rmse_test #test setinin tahmin başarısı

1.7815996615334497

# Model Tuning / Model Doğrulama

Çoklu doğrusal regresyon modelinde B0'dan başka dışsal parametre yok (B0 da değil aslında). Burada model tuning işlemini model doğrulama olarak ele alacağız. Model doğrulamanın görmüş olduğumuz yöntemlerce mantığını ele alıp değerlendirip bu modelde tuningi doğrulama olarak uygulayacağız.

In [27]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [28]:
X = df.drop('sales', axis=1)
y = df["sales"]
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=144)
lm = LinearRegression() 
model = lm.fit(X_train, y_train)

In [29]:
np.sqrt(mean_squared_error(y_train, model.predict(X_train)))

1.6748559274650712

In [30]:
np.sqrt(mean_squared_error(y_test, model.predict(X_test)))

1.6640263686701038

In [31]:
model.score(X_train, y_train)

0.8971614078663419

In [32]:
# burada modele cross validation yaparak gerçekçi mse değerine ulaşacağız.
# aşağıda 10 katlı bir cv yapıp bunun ortalamasını alıyoruz ve bu ortalama güvenilir (valide edilmiş) mse değerine ulaşıyoruz

In [33]:
cross_val_score(model, X, y, cv = 10, scoring = "r2").mean()

0.8853562237979616

### --------------------------------------------------------------------------------------------------------
Aşağıdaki gibi direkt score fonksiyonu ile bulunan mse değeri ortalamadan farklı, burada ortalama train grupları için  daha doğru bir mse değeridir

In [34]:
cross_val_score(model, X_train, y_train,
                cv = 10, 
                scoring = "r2").mean() # bu da train için 10 validasyon yaparak bulduğumuz mse değeri

0.8733783298422939

In [35]:
model.score(X_train, y_train)

0.8971614078663419

### --------------------------------------------------------------------------------------------------------

# Asıl Önemli Doğrulama Kısmı

In [36]:
## train grubunun tek işlemli ve 10 validasyonlu r2 değerlerinin karşılaştırılması

In [37]:
model.score(X_train, y_train) # r2 değeri

0.8971614078663419

In [38]:
cross_val_score(model, X_train, y_train,
                cv = 10, 
                scoring = "r2").mean() # bu da train grubu için 10 validasyon yaparak bulduğumuz r2 değeri

0.8733783298422939

In [39]:
## burada daha güvenilir olan değer validasyonla bulunmuş değer yani 0.84

In [40]:
## burada train için validasyon yaparken adımları görüyoruz

In [41]:
-cross_val_score(model, X_train, y_train,
                cv = 10, 
                scoring = "neg_mean_squared_error") # datayı 10'a ayırıp train hatası değerleri oluşturuldu (r2)

array([3.77011117, 1.38904597, 1.31506551, 3.32109589, 8.82506973,
       2.37926645, 2.2872061 , 1.05714426, 2.99532621, 3.15248307])

In [42]:
np.sqrt(-cross_val_score(model, X_train, y_train,
                cv = 10, 
                scoring = "neg_mean_squared_error")) # elimizde train seti için 10 grubun rmse değeri oldu

array([1.94167741, 1.17857794, 1.14676306, 1.82238742, 2.97070189,
       1.5424871 , 1.51235118, 1.02817521, 1.73070108, 1.77552332])

In [43]:
np.sqrt(-cross_val_score(model, X_train, y_train,
                cv = 10, 
                scoring = "neg_mean_squared_error")).mean() # burada da bu 10 grubun rmse'lerinin ortalamasını alıyoruz

1.6649345607872932

In [44]:
np.sqrt(mean_squared_error(y_train, model.predict(X_train))) # 1 adet train seti için rmse değeri bulmuştuk önceden

1.6748559274650712

#### Yukarda adım adım nasıl rmse ortalamasına validasyonla ulaştığımızı ve valide edilmiş rmse ile tekli rmse'nin farkını görüyoruz

#### Validasyon yaparken verinin farklı parçalarından rmse değerleri alınarak ortalaması alınıyor.

### ---------------------------------------------------------------------------------------------------------

In [45]:
##

In [46]:
## Son olarak da asıl kullanacağımız test hatasının valide edilmiş değerini görüyoruz
## Bu değer bize ml algoritmamızın gerçek hata payını veriyor diyebiliriz.

In [47]:
np.sqrt(mean_squared_error(y_test, model.predict(X_test))) # test hatası

1.6640263686701038

In [48]:
np.sqrt(-cross_val_score(model, X_test, y_test,
                cv = 10, 
                scoring = "neg_mean_squared_error")).mean() # valide edilmiş test hatası değeri yani bunu kullanılırız ölçümde

1.7399924960346642

In [49]:
# train hatası ve test hatası birbirinden farklılaşır, eğitim hatası test hatasının kötü bir tahmincisidir.